# Подключение

In [1]:
import pyodbc

In [51]:
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=10.199.13.60;DATABASE=rway;UID=vkomarnitskii;PWD=Rway1')
cursor = conn.cursor()


## Pandas

In [3]:
import pandas as pd
from math import isnan

Тут можно установить параметры отображения для pandas. Сколько показывать строк и столбцов соответственно.

In [4]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 150

### Отладка


In [119]:
def benchmark(func):
    """
    Декоратор, выводящий время, которое заняло
    выполнение декорируемой функции
    """
    import time
    
    def wrapper(*args, **kwargs):
        t = time.process_time()
        res = func(*args, **kwargs)
        print(f'{func.__name__} выполнилась за время {time.process_time() - t}s')
        return res
    
    return wrapper

## Необходимые функции

Позволяет получить список характеристик для всех предложений, указанных в list_of_keys. Предложения указываются в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

Преобразование из байтов в такую строку делается следующим образом:
```python
# type(x) == bytes
result = '0x' + x.hex().upper()
```

In [53]:
@benchmark
def get_harks_by_object_keys(task_id):
    """
    :param task_id: id задачи
    :type task_id: str
    :return: DataFrame, в котором указаны характеристики и их значения для всех объектов из задачи task_id
    """
    
#     in_expr = f'({", ".join(list_of_keys)})'
    query = '''
    SELECT
    t_har.Наименование,
    CASE 
        WHEN t.Значение_Тип = 04 THEN CAST(t.Значение_Дата AS varchar)
        WHEN t.Значение_Тип = 03 THEN CAST(t.Значение_Число AS varchar)
        WHEN t.Значение_Тип = 05 THEN t.Значение_Строка
        WHEN t.Значение_Тип = 01 THEN 
        CASE
            WHEN t.Значение = 0x869CEA7372E5CA3E468F18025D610C0B THEN 'Да'
            WHEN t.Значение = 0xB0CD188371B9A88E431B073454A9489F THEN 'Нет'
            ELSE 'НетДанных'
        END
        WHEN t.Значение_Тип = 08 THEN 
        COALESCE(
            (SELECT Наименование FROM [rway].[Справочник].[ДополнительныеЗначенияХарактеристик] t_dop WHERE t_dop.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ТипыОбъектовНедвижимости] t_types WHERE t_types.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыСделки] t_types_1 WHERE t_types_1.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ЛогическиеЗначения] t_logic WHERE t_logic.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Перечисление].[ТипыВерифицированности] t_verif WHERE t_verif.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ПодСегменты] t_podseg WHERE t_podseg.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[Застройщики] t_zastr WHERE t_zastr.Ссылка = t.Значение),
            (SELECT Наименование FROM [rway].[Справочник].[ОстановкиОбщественногоТранспорта] t_stations WHERE t_stations.Ссылка = t.Значение)                  
        )
    END
        AS Значение,
    t.Объект AS Ссылка
FROM [rway].[РегистрСведений].[ЗначенияХарактеристик] t
    INNER JOIN [rway].[ПВХ].[Характеристики] t_har
        ON t_har.[Ссылка] = t.[Характеристика]
    INNER JOIN [rway].[РегистрСведений].[ПредложенияЗадач] reg_task
        ON reg_task.Предложение = t.Объект
        AND reg_task.Задача in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{}')
    '''.format(task_id)
    
    cursor.execute(query)
#     data_df = pd.read_sql(sql=query, con=conn)

    # Преобразование данных, которые вернул запрос в список списков, одновременно все байты преобразуются в hex
    data = list(map(lambda x: list(x), cursor.fetchall()))
    data_df = pd.DataFrame(data, columns=list(map(lambda x: x[0], cursor.description)))
    # Создание словаря для DataFrame
    return data_df.pivot(index='Ссылка', columns='Наименование', values='Значение')

    

Функция позволяет получить все предложения по конкретной задаче. Задача указывается в виде строки `hex` в верхнем регистре, первыми символами которой являются `0x`.

In [164]:
@benchmark
def get_base_info_by_task_id(task_id, offers_count=None):
    """
    :param task_id: id задания
    :param offers_count: Количество предложений для выбора из базы (пустое для выбора всех предложений)
    :type task_id: str
    :return: DataFrame, содержащий все поля, которые можно получить из таблицы "ПредложенияЗадач"
    """
    
    # Запрос к дате задания
    date = cursor.execute("SELECT Дата FROM [rway].[Документ].[Задание] WHERE КодЗадания = '{}'".format(task_id)).fetchone()
    
    top = '' if not offers_count else 'TOP {}'.format(offers_count)
    query = '''
    SELECT {top}
          t.Код,
          t.Ссылка,
          t_task._Fld198 AS КодЗадачи,
          t_source.Наименование AS Источник,
          t.АдресAhunter,
          t.АктуальнаяСсылкаИсточника,
          t.ДатаПересмотраЭкспозиции,
          t.ДатаПроверкиАктуальности,
          t.ДатаРазмещения,
          t.Город,
          t.Описание,
          (SELECT Наименование FROM [rway].[Справочник].[Подсегменты] WHERE Ссылка = t.Подсегмент) AS Подсегмент,
          (SELECT Наименование FROM [rway].[Справочник].[Сегменты] WHERE Ссылка = t.Сегмент) AS Сегмент,
          (SELECT Наименование FROM [rway].[Справочник].[СубъектыРФ] WHERE Ссылка = t.Субъект) AS Субъект,
          t.СсылкаИсточника,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыРынка] WHERE Ссылка = t.ТипРынка) AS ТипРынка,
          (SELECT Значение FROM [rway].[Перечисление].[ТипыСделки] WHERE Ссылка = t.ТипСделки) AS ТипСделки
    FROM [rway].[Справочник].[ПредложенияОбъектовНедвижимости] t
        JOIN [rway].[РегистрСведений].[ПредложенияЗадач] t_offer
            ON Ссылка = t_offer.[Предложение]
        JOIN [rway].[dbo].[_Task62] t_task
            ON t_task.[_IDRRef] = t_offer.[Задача] AND t_task.[_IDRRef] in (
            SELECT
                task._IDRRef AS Ссылка
            FROM [rway].[dbo].[_Task62] task
            JOIN [rway].[Документ].[Задание] tasks
                ON task._Fld192RRef = tasks.Ссылка
            JOIN [rway].[Справочник].[ТипыЗадач] types
                ON types.Ссылка = task._Fld231RRef AND types.Наименование = 'Импорт'
            WHERE tasks.КодЗадания = '{task_id}')
        JOIN [rway].[Справочник].[Источники] t_source
            ON t_source.Ссылка = ИсточникИнформации

    '''.format(top=top, task_id=task_id)
    
    cursor.execute(query)    
    data = cursor.fetchall()
    # Те же самые преобрзования, как в функции с характеристиками
    df = pd.DataFrame(map(lambda x: list(x), data))
    df.columns = list(map(lambda x: x[0], cursor.description))
    df['КодЗадания'] = task_id
    df['ДатаЗадания'] = date[0]
#     df = pd.read_sql(sql=query, con=conn)
    return df
    

# Боевой запуск

In [165]:
# Сбор основных данных из "ПредложенияОбъектовНедвижимости"
base_df = get_base_info_by_task_id('0001-0405')

get_base_info_by_task_id выполнилась за время 17.453125s


In [166]:
base_df


,Код,Ссылка,КодЗадачи,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,КодЗадания,ДатаЗадания
0,190725W00115200,b'\xab\x1b\xb3\x02D\x05\x9d\x1dK\xcb\xb4\xd0\x...,0001-0405-0046,RealtyVision.ru,"обл Иркутская, г Иркутск, ул Пискунова, дом 148/2",,2019-07-25 00:00:00,2019-07-25 14:53:59,2019-07-25 00:00:00,Иркутск,нет данных,None,Коммерческая Недвижимость,Иркутская обл,https://www.realtyvision.ru/kommercheskaya-ned...,Вторичный,None,0001-0405,2019-07-24 13:52:21
1,190725W00271869,b'\xbbw\t\x0c\x855\xfe\xf4L\xccrQ\xa99\xca\x81',0001-0405-0054,MOVE.ru,"Респ Крым, р-н Белогорский, г Белогорск, ул Би...",,2019-07-25 00:00:00,2019-07-25 16:08:47,2019-07-22 00:00:00,Белогорск,Учacтoк 16 сотoк в чеpте городa в чаcтной собс...,ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_psn_kr...,Вторичный,None,0001-0405,2019-07-24 13:52:21
2,190725W00406473,b'\xbb|7\xc2\xd7\xbd_u@y\xf1\x036\xbc\xd4\x96',0001-0405-0054,MOVE.ru,"Респ Крым, г Симферополь, пр-кт Победы, дом 48А",,2019-07-25 00:00:00,2019-07-25 16:51:50,2019-07-20 00:00:00,Симферополь,Пpодaется Зданиe под коммерчеcкие цeли в центp...,ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_psn_pl...,Вторичный,None,0001-0405,2019-07-24 13:52:21
3,190725W00271617,"b'\xbb\x93""w&~\xbf\xdfE\x83\xdd\xf77\xf3\t\x1b'",0001-0405-0063,Олимп - АН,,,2019-07-25 00:00:00,2019-07-25 16:31:53,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Краснодарский край,http://olymprealty.ru/sale/397446/nedvizhimost...,Вторичный,None,0001-0405,2019-07-24 13:52:21
4,190725W00405629,b'\xbb\x9e<\xf0\x88\xd1f\xff@\xa2\x92\xbep\x88...,0001-0405-0054,MOVE.ru,"Респ Крым, р-н Белогорский, с Кизиловка",,2019-07-25 00:00:00,2019-07-25 16:50:07,2019-07-23 00:00:00,,Прoмышленнo складскaя база ПМК-11 в Kизиловoм....,производственно-складской,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_psn_pl...,Вторичный,None,0001-0405,2019-07-24 13:52:21
5,190725W01084979,"b'\xbb\xa0\xa8q\xedp\xea\xf4O\xcc,\xa7\x12\xb5w;'",0001-0405-0054,MOVE.ru,"Респ Крым, г Симферополь, ул Совхозная, дом 4Д",,2019-07-25 00:00:00,2019-07-25 16:31:26,2019-07-23 00:00:00,Симферополь,"Пoмeщениe cвобoднoго назначения , oтдельнaя вх...",ПСН,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/sdaetsya_psn_plos...,Вторичный,None,0001-0405,2019-07-24 13:52:21
6,190725W00627231,"b'\xbb\xac|,\xd4+\xba\xeb@\x04\xd6a\x82\xaf\xb...",0001-0405-0054,MOVE.ru,"Респ Крым, г Феодосия, пер Школьный, дом 8",,2019-07-25 00:00:00,2019-07-25 17:50:32,2019-07-21 00:00:00,Феодосия,Частичка души в Коктебеле «У каждого есть таки...,гостиничный,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_15-kom...,Вторичный,None,0001-0405,2019-07-24 13:52:21
7,190725W00405709,b'\xbb\xc8<\xbe!\xf7\x9a<K_+\x06kh[c',0001-0405-0054,MOVE.ru,"Респ Крым, г Ялта, ул Московская, дом 45",,2019-07-25 00:00:00,2019-07-25 16:50:11,2019-07-20 00:00:00,Ялта,ID:75. Предлагаем к приобретению коммерческую ...,офисный,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_ofis_p...,Вторичный,None,0001-0405,2019-07-24 13:52:21
8,190725W00976531,b'\xbb\xdf\x1a-\x17b\x94|J\x80\xabMz\xb3\x84\xf0',0001-0405-0054,MOVE.ru,"Респ Крым, г Симферополь, ул Объездная, дом 10Б",,2019-07-25 00:00:00,2019-07-25 16:06:02,2019-07-23 00:00:00,Симферополь,Срочно ! Продам сдвоенный бутик на радиорынке ...,торговый,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_kommer...,Вторичный,None,0001-0405,2019-07-24 13:52:21
9,190725W00299627,b'\xbb\xef\xe2\xf5r\x03\x8e\xeeJ\x93\xea\x9c\x...,0001-0405-0054,MOVE.ru,"Респ Крым, г Евпатория",,2019-07-25 00:00:00,2019-07-25 16:29:25,2019-07-23 00:00:00,Евпатория,Hомеpной фoнд панcионата сoстaвляют 23 номepa:...,гостиничный,Коммерческая Недвижимость,Крым Респ,https://krim.mov

In [ ]:
# Сбор характеристик по ссылкам из основного DataFrame base_df
har_df = get_harks_by_object_keys('0001-0405')

In [ ]:
har_df

In [115]:
# Собираем обе таблицы в одну большую
result_df = pd.merge(base_df, har_df, on='Ссылка', how='left')

In [116]:
result_df

,Код,Ссылка,КодЗадачи,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,КодЗадания,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж
0,190725W00628329,b'\x97\x18r\x92\xd3\x90\xf1\rB\x0f)?F\xb0\x12\...,0001-0405-0054,MOVE.ru,"Респ Крым, г Ялта, ул Пушкинская, дом 26",,2019-07-25 00:00:00,2019-07-25 17:49:57,2019-07-18 00:00:00,Ялта,"Продaётся тоpгoвое помещениe в центpе Ялты, у...",торговый,Коммерческая Недвижимость,Крым Респ,https://krim.move.ru/objects/prodaetsya_kommer...,None,None,0001-0405,NaN,NaN,Пушкинская ул,,NaN,NaN,NaN,NaN,34.154856,44.490710,NaN,2019-07-25 17:49:57,"Продам коммерческую недвижимость, Ялта, Пушкин...",NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,торговая площадь,NaN,0.000000,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Продавец,кв.м.,руб.,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,None,NaN,13000000.000000,NaN,NaN,
1,190725W00143303,b'\x81J\xa7\x9d-\x92\x9b\x8bL\xa3D_#\x9b|\x8c',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 15:30:38,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 15:30:38,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
2,190725W00088154,b'\x82\x1b\x8dExG|pBB1\xf6\xc6~\xfes',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:31:52,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:31:52,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
3,190725W00113236,b'\x82<x\xf5\xb0\xa2x\xd6N\xfd#b\xe1\xa2\xfa%',0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:57:49,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:57:49,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,,кв.м.,None,NaN,NaN,NaN,Без Проведения Торгов,NaN,NaN,нет данных,NaN,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,
4,190725W00083083,b'\x82N\x91{\xf0\xaf\x10\xf7M\xa4\xe7\xcb\xc5n...,0001-0405-0011,КУПИ.ру,обл Магаданская,,2019-07-25 00:00:00,2019-07-25 14:28:08,2019-07-25 00:00:00,,нет данных,None,Коммерческая Недвижимость,Магаданская обл,https://qp.ru/?standart=true,Вторичный,None,0001-0405,NaN,NaN,,,",",NaN,NaN,NaN,NaN,NaN,NaN,2019-07-25 14:28:08,,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN

In [139]:
@benchmark
def make_test():
    new_df = pd.DataFrame(columns=set(result_df.loc[:, 'Источник']), index=list(result_df.columns.fillna(0)) + ['Всего'])
    nan = float('nan')
    for source in set(result_df['Источник']):
        filtered = result_df[result_df['Источник'] == source]
        new_df.loc['Всего', source] = len(filtered)
        count_s = filtered.replace('', nan).replace('НетДанных', nan).replace('0.000000', nan).count()
        for k, v in count_s.items():
            new_df.loc[k, source] = v * 100 / new_df.loc['Всего', source]
        new_df.loc['КодЗадания', source] = list(filtered['КодЗадания'])[0]
        new_df.loc['КодЗадачи', source] = list(filtered['КодЗадачи'])[0]

    return new_df

Функция, в которой и происходит тестирование.

In [140]:
test = make_test().T

make_test выполнилась за время 34.4375s


Выгрузка данных в `csv` файл для удобного просмотра в Excel

In [141]:
test.to_csv('test_result.csv', header=list(test.columns), encoding='cp1251', sep=';')
test

,Код,Ссылка,КодЗадачи,Источник,АдресAhunter,АктуальнаяСсылкаИсточника,ДатаПересмотраЭкспозиции,ДатаПроверкиАктуальности,ДатаРазмещения,Город,Описание,Подсегмент,Сегмент,Субъект,СсылкаИсточника,ТипРынка,ТипСделки,КодЗадания,3.Муниципальное Образование,5.Внутригородской Район,7.Улица,8.Дом,Адрес (временная),Ближайшая Станция Метро,Водоснабжение,Высота Потолков,ГеоКод Долгота,ГеоКод Широта,Год Постройки,Дата Сбора Информации,Заголовок,Интернет,Класс Объекта,Количество Комнат,Количество Этажей,Кондиционирование,Лифты,Материал Наружных Стен,Минут До Ближайшей Станции Метро,Мощность Электричества,НДС,Назначение Объекта Предложения,Наличие Отделки Помещений,Общая Площадь Предложения,Ориентир,Отдельный Вход,Охрана Предложения,"Парковка Наземная, Количество м/м",Планировка Помещения,Площадь Всего Объекта,Площадь Земельного Участка Объекта,Пожаротушение,Попадает в полигон,Предмет Сделки,Продавец,Размерность Площади,Размерность Стоимости,Расположение Относительно Первой Линии Домов,Расстояние До Ближайщей Станции Метро,Система Вентиляции,Способ Реализации,Статус Строительства,Субъект РФ,Телефон Продавца,Температурный Режим Склада,Тип Объекта,Тип Парковки,Тип объекта недвижимости,Управляющая Компания,Цена,Цена Предложения За 1 кв.м.,Электричество,Этаж,Всего
Необходимая недвижимость,100,100,0001-0405-0048,100,99.297,0,100,100,100,91.2127,100,99.297,100,100,100,100,100,0001-0405,0.878735,0.702988,97.3638,78.2074,0.175747,0,0,0,0,0,0,100,0,0,8.08436,0,46.2214,0,0,0,0,0,0,98.9455,0,100,0,0,0,0,0,0,0,0,0.878735,0,100,100,100,0,0,0,100,0,0,100,0,0,0,2.9877,0,100,99.297,0,48.6819,569
Buy business portal,100,100,0001-0405-0070,100,98.6425,100,100,100,100,86.2745,100,99.5475,100,99.6983,100,100,100,0001-0405,0.15083,0,80.9955,53.092,1.80995,0,0,0,0,0,2.5641,100,100,0,1.5083,0,19.3062,0,0,0,0,0,0,99.5475,0,98.3409,0,0,0,0,0,0,0,0,0.452489,0,100,99.3967,100,0,0,0,100,0,0,100,0,1.05581,0,4.67572,0,99.6983,96.6817,0,25.0377,663
Адвекс - АН,100,100,0001-0405-0022,100,100,0,100,100,100,100,100,100,100,100,100,100,100,0001-0405,0,0,100,94.7368,0,0,94.7368,0,100,100,0,100,100,0,0,0,63.1579,0,0,0,0,0,0,100,89.4737,100,0,89.4737,0,0,0,0,0,0,0,0,100,100,100,0,0,0,100,0,0,100,94.7368,0,0,0,0,100,100,94.7368,47.3684,19
Издательский дом Ярмарка,100,100,0001-0405-0044,100,100,0,100,100,100,75,100,100,100,100,100,100,100,0001-0405,0,0,75,50,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,100,0,100,0,0,0,0,0,0,0,0,0,0,100,100,100,0,0,0,100,0,0,100,0,0,0,50,0,100,100,0,0,4
ИС Центр,100,100,0001-0405-0042,100,98.4343,0,100,100,100,90.776,100,99.7958,100,100,100,100,100,0001-0405,0.170184,0.170184,89.5848,57.8965,1.05514,0,0,0,0,0,0,100,100,0,0.816882,0,6.05854,0,0,0,0,0,0,99.7277,0,39.3125,0,0,0,0,0,0,0,0,0.204221,0.0340368,100,0,100,0,0,0,100,0,0,100,0,0,0,7.3179,0,100,39.3125,0,16.4738,2938
AVITO.ru,100,100,0001-0405-0006,100,99.9329,0,100,100,100,94.243,99.9985,100,100,100,100,100,100,0001-0405,0.802826,0.675833,92.9891,80.7249,1.09184,0,0,0,97.9068,97.9068,0,100,100,0,6.68535,0,11.9811,0,0,0,0,0,0,99.9825,0,99.9942,0,0,0,0,0,0,0,0,0.826181,0.442284,100,99.9971,100,0,0,0,100,0,0,100,0,0,0,1.88591,0,99.9241,99.8511,0,42.3352,68508
Zdanie.info,100,100,0001-0405-0027,100,99.9865,0,100,100,100,99.314,100,99.9865,100,100,100,100,100,0001-0405,0.295938,0.269034,98.332,79.6341,0.201776,0,0,0,0,0,0,100,100,0,9.48345,0,25.1278,0,0,0,0,0,0,99.9865,0,98.6145,0,0,0,0,0,0,0,0,0.295938,0.094162,100,96.0721,100,0,0,0,100,0,0,100,0,0,0,12.6446,0,84.9206,83.7638,0,29.4861,7434
Granta24,100,100,0001-0405-0017,100,100,0,100,100,100,99.2,100,99.8,100,100,100,100,100,0001-0405,0,0,100,3.2,0,0,0,0,95.4,95.6,0,100,100,0,0.4,0,100,0,0,82.6,0,0,0,99.8,0,100,0,0,0,0,0,0,0,0,0,0,100,100,100,0,0,0,100,0,0,100,0,0,0,0.2,0,100,100,0,98,500
Олимп - АН,100,100,0001-0405-0063,100,0,0,100,100,100,0,100,0,100,100,100,100,0,0001-0405,0,0,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,100,0,0,0,0,0,100,0,0,100,0,0,0,0,0,0,0,0,0,5
ЦИАН,100,100,0001-0405-0001,100,99.9562,0,100,100,100,9

## Для тестов


In [155]:
cursor = conn.cursor()
query = '''
SELECT
                Дата,
                *
            FROM [rway].[Документ].[Задание] task
    
'''
# WHERE _FLD198 = '0001-0402-0001'

cursor.execute(query)
data = cursor.fetchall()
df = pd.DataFrame(map(lambda x: list(map(lambda y: '0x' + y.hex().upper() if type(y) == bytes else y, x)), data))
df.columns = list(map(lambda x: x[0], cursor.description))
df


,Дата,Ссылка,ВерсияДанных,ПометкаУдаления,Дата,Номер,Проведен,ДатаЗавершенияЗадания,ДатаНачалаЗадания,ЖелаемоеКоличество,КодЗадания,КоличествоИсточники,КоличествоСегменты,КоличествоСубъекты,КритерийВерификацииДляЭкспорта,Наименование,Ответственный,ОчиститьНеверифицированные,ПредметОбработки,ПредыдущееЗадание,Проект,Состояние,Статус,ТекстУсловияВерификацииSQL,УдалитьЗадание,ШаблонСценария,ШаблонУровнейВерификации,ШаблонХарактеристик,ЭкспортироватьВАрхивПолучатель,ЭкспортироватьВАрхив,ВерсияARH
0,2019-07-24 13:52:21,0xBBBA30E1716029C711E9AE011C42A8FB,0x00000000043BD9BB,0,2019-07-24 13:52:21,23,0,2019-07-31 00:00:00,2019-07-24 00:00:00,300000,0001-0405,73,1,93,0xBB9A30E1716029C711E955ECE8AA31D4,коммерческая вторичная к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0xBBBA30E1716029C711E9A85EDF78B9F2,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBBB030E1716029C711E9971813456627,0x80FB30E1716029C711E8E35B7F26B37A,0xBBAC30E1716029C711E9928D38D72A19,ROL Эксплуатация,1,0
1,2019-07-25 15:48:42,0xBBBA30E1716029C711E9AEDA87E5C5DD,0x00000000042BDE5E,0,2019-07-25 15:48:42,24,0,2019-07-31 00:00:00,2019-07-25 00:00:00,20000,0001-0406,76,1,85,0xBB9A30E1716029C711E955ECE8AA31D4,зу вторичная к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0xBBBA30E1716029C711E9A85ED736875B,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBB9A30E1716029C711E926D5CC8B7D94,0xBB9A30E1716029C711E9372CE9270C58,0xBB9A30E1716029C711E95C32BD197E1F,ROL Archive,0,0
2,2019-07-28 15:32:54,0xBBBB30E1716029C711E9B133CD754E42,0x0000000004331601,0,2019-07-28 15:32:54,25,0,2019-07-31 00:00:00,2019-07-28 00:00:00,0,0001-0407,76,1,85,0xBB9A30E1716029C711E955ECE8AA31D4,готовый бизнес к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0x00000000000000000000000000000000,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8ED2D56DC8116F1349D0AD43595892BB,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0xBBB530E1716029C711E99B21F22F0E2C,0xBB9A30E1716029C711E938446B8C6EC2,0xBBB030E1716029C711E9963FA27FB1FB,ROL Archive,1,0
3,2019-07-28 18:49:35,0xBBBB30E1716029C711E9B14F4BDE76D6,0x00000000043315FC,0,2019-07-28 18:49:35,26,0,2019-07-31 00:00:00,2019-07-28 00:00:00,2500,0001-0408,1,1,85,0xBB9A30E1716029C711E955ECE8AA31D4,коммерческая торги к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0x00000000000000000000000000000000,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8ED2D56DC8116F1349D0AD43595892BB,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0x80DD30E1716029C711E7CB78B41BB757,0x80F630E1716029C711E8E1B589261D9C,0xBB9A30E1716029C711E96683AF339C27,ROL Archive,1,0
4,2019-07-28 18:50:03,0xBBBB30E1716029C711E9B14F5CD18A54,0x00000000042A7814,0,2019-07-28 18:50:03,27,0,2019-07-31 00:00:00,2019-07-28 00:00:00,500,0001-0409,1,0,85,0xBB9A30E1716029C711E955ECE8AA31D4,зу торги к 31.07,0xBBAC30E1716029C711E99286A96DF4AC,0,0xAB47107B44B1669011E990D14C17FFEE,0x00000000000000000000000000000000,0x80BC00155D645E0311E72A9031B82F51,0x00000000000000000000000000000000,0x8A2B026DDB6080A5485EA0659DE117D3,"AND (ISNULL(T3.[Уровень9], ЗНАЧЕНИЕ(Перечислен...",0,0x80F630E1716029C711E8B66229012248,0xBB9A30E1716029C711E935097625DB69,0xBBAC30E1716029C711E99289027F858B,ROL Archive,0,0


In [163]:
cursor.execute("SELECT Дата FROM [rway].[Документ].[Задание] WHERE КодЗадания = '{}'".format('0001-0405')).fetchone()

('2019-07-24 13:52:21', )